In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd 
%matplotlib inline 
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

#modelling 

from sklearn.metrics import mean_Squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from 